### Analysis of factors that affect student's quality of academic performance

1- Age
2- Gender
3 - Geographical belongingness
4 - Ethnicity
5 - Marital status
6 - socioeconomic status (SES)
7 - parents’ education level
8 - parental profession
9 - language
10 - income
11 - religious affiliations


# Importação das bibliotecas

In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import locale
import dask.dataframe as dd
from utils import load_parquets, info_sum_isna
from scipy.stats import ranksums
from scipy import stats

# Carregando os dados

### Dataset information:



In [32]:
year = 2020
path = f'../datasets/integrated_datas_{year}.parquet.gzip'
data = dd.read_parquet(path, ignore_metadata_file=True)

Testando se veio de uma distribuicao normal

In [33]:
# testing_normal_distribution(mulher_math_pb, 'NU_NOTA_MT')

In [34]:
# testing_normal_distribution(homem_math_pb, 'NU_NOTA_MT')

Testing normal distribution for math grades (male and female) for all Brazilian states to decide which statistical test would be suitable

In [35]:
# def ploting_distribution_female_male_per_state(state, course):
#     dataset_women = mulher_2020[mulher_2020['SG_UF_RESIDENCIA'] == state]
#     dataset_man = homem_2020[homem_2020['SG_UF_RESIDENCIA'] == state]
#     fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=True)
#     ax1 = sns.histplot(dataset_women[course], ax=ax1).set(title="Math grades distribution 'W'", xlabel='Math grades W', ylabel='density')
#     ax2 = sns.histplot(dataset_man[course], ax=ax2).set(title="Math grades distribution 'M'",  xlabel='Math grades M', ylabel='density')
#     plt.show()

## Testes estatisticos para 3 variavies

### 1. Raca

In [36]:
data['TP_COR_RACA'].value_counts().compute()

3    1157403
1     974067
2     309833
4      56786
0      49395
5      13820
Name: TP_COR_RACA, dtype: int64

In [37]:
def replace_ethnicity(value):
    if value == 1:
        return "White"
    elif value == 2:
        return "Black"
    elif value == 3:
        return "Brown"
    elif value == 4:
        return "Yellow"
    elif value == 5:
        return "Indigenous"

In [38]:
data['TP_COR_RACA'] = data.TP_COR_RACA.apply(replace_ethnicity)


/Users/chrystinne/.pyenv/versions/3.9.4/envs/venem/lib/python3.9/site-packages/dask/dataframe/core.py:4132: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('TP_COR_RACA', 'object'))

  warnings.warn(meta_warning(meta))


In [39]:
data['TP_COR_RACA'].value_counts().compute()

Brown         1157403
White          974067
Black          309833
Yellow          56786
Indigenous      13820
Name: TP_COR_RACA, dtype: int64

In [40]:
black = data[(data['TP_COR_RACA'] == 'Black')]
white = data[(data['TP_COR_RACA'] == 'White')]
brown = data[(data['TP_COR_RACA'] == 'Brown')]
yellow = data[(data['TP_COR_RACA'] == 'Yellow')]
indigenous = data[(data['TP_COR_RACA'] == 'Indigenous')]

In [55]:
def has_normal_distribution(dataset, course):
    k2, p = stats.normaltest(dataset[course])
    alpha = 0.05
    # print("p = {:g}".format(p))
    if p < alpha:  # null hypothesis: x comes from a normal distribution
        print("The null hypothesis can be rejected")
        return False
    else:
        print("The null hypothesis cannot be rejected")
        return True

In [42]:
states = list(data['SG_UF_RESIDENCIA'].unique())

In [43]:
has_normal_distribution(black[black['SG_UF_RESIDENCIA'] == 'PB'], 'NU_NOTA_MT')

False

In [44]:
def testing_normal_distribution_all_states(course):
    number_of_normal_distributions = 0
    for state in states:
        print(state)
        black_state = black[black['SG_UF_RESIDENCIA'] == state]
        white_state = white[white['SG_UF_RESIDENCIA'] == state]
        yellow_state = yellow[yellow['SG_UF_RESIDENCIA'] == state]
        indigenous_state = indigenous[indigenous['SG_UF_RESIDENCIA'] == state]
        brown_state = brown[brown['SG_UF_RESIDENCIA'] == state]
        result_black_dist = has_normal_distribution(black_state, 'NU_NOTA_MT')
        result_brown_dist = has_normal_distribution(brown_state, 'NU_NOTA_MT')
        result_yellow_dist = has_normal_distribution(yellow_state, 'NU_NOTA_MT')
        result_indigenous_dist = has_normal_distribution(indigenous_state, 'NU_NOTA_MT')
        result_white_dist = has_normal_distribution(white_state, 'NU_NOTA_MT')
        if (result_black_dist | result_brown_dist | result_yellow_dist | result_indigenous_dist | result_white_dist):
            number_of_normal_distributions+= 1
            print('_____________________________ normal distribution _____________________________ ')
    return number_of_normal_distributions


In [56]:
black_state = black[black['SG_UF_RESIDENCIA'] == 'SC']
white_state = white[white['SG_UF_RESIDENCIA'] == 'SC']
yellow_state = yellow[yellow['SG_UF_RESIDENCIA'] == 'SC']
indigenous_state = indigenous[indigenous['SG_UF_RESIDENCIA'] == 'SC']
brown_state = brown[brown['SG_UF_RESIDENCIA'] == 'SC']

has_normal_distribution(black_state, 'NU_NOTA_MT') #abnormal
has_normal_distribution(brown_state, 'NU_NOTA_MT') #abnormal
has_normal_distribution(yellow_state, 'NU_NOTA_MT') #abnormal
has_normal_distribution(indigenous_state, 'NU_NOTA_MT') #normal distribution
has_normal_distribution(white_state, 'NU_NOTA_MT') #abnormal

The null hypothesis can be rejected
The null hypothesis can be rejected
The null hypothesis can be rejected
The null hypothesis cannot be rejected
The null hypothesis can be rejected


False

In [58]:
black_state = black[black['SG_UF_RESIDENCIA'] == 'RR']
white_state = white[white['SG_UF_RESIDENCIA'] == 'RR']
yellow_state = yellow[yellow['SG_UF_RESIDENCIA'] == 'RR']
indigenous_state = indigenous[indigenous['SG_UF_RESIDENCIA'] == 'RR']
brown_state = brown[brown['SG_UF_RESIDENCIA'] == 'RR']

has_normal_distribution(black_state, 'NU_NOTA_MT') #abnormal
has_normal_distribution(brown_state, 'NU_NOTA_MT') #abnormal
has_normal_distribution(yellow_state, 'NU_NOTA_MT') #normal 
has_normal_distribution(indigenous_state, 'NU_NOTA_MT') #abnormal
has_normal_distribution(white_state, 'NU_NOTA_MT') #abnormal

The null hypothesis can be rejected
The null hypothesis can be rejected
The null hypothesis cannot be rejected
The null hypothesis can be rejected
The null hypothesis can be rejected


False

In [45]:
testing_normal_distribution_all_states('NU_NOTA_MT')

RS
PB
BA
AL
PA
TO
SP
CE
AM
SE
MG
MA
PI
PE
MT
RJ
GO
RN
ES
AP
DF
SC
The null hypothesis cannot be rejected
SC
_____________________________ normal distribution _____________________________ 
PR
RR
The null hypothesis cannot be rejected
RR
_____________________________ normal distribution _____________________________ 
RO
MS
AC


2

In [46]:
def perform_Kruskal_Wallis_per_state(state, course):
   return stats.kruskal(black[black['SG_UF_RESIDENCIA'] == state][course], white[white['SG_UF_RESIDENCIA'] == state][course], brown[brown['SG_UF_RESIDENCIA'] == state][course], yellow[yellow['SG_UF_RESIDENCIA'] == state][course], indigenous[indigenous['SG_UF_RESIDENCIA'] == state][course])

In [47]:
print(perform_Kruskal_Wallis_per_state('PB', 'NU_NOTA_MT'))

KruskalResult(statistic=1869.501424945442, pvalue=0.0)


Interpreting the results:

The Kruskal-Wallis Test uses the following null and alternative hypotheses:

The null hypothesis (H0): The median score in math is equal across all ethnicty groups.

The alternative hypothesis: (Ha): The median score in math is not equal across all ethnicty groups.

In this case, the test statistic is 7749.868515864906 and the corresponding p-value is 0.0. Since this p-value is less than 0.05, we can reject the null hypothesis that the median score in math is the same for all five ethnicity groups. We have sufficient evidence to conclude that the ethnicity leads to statistically significant differences in median scores in math for the ENEM exam.